## Recursive inference (multi-step) for BSCTRFM models

### use time series instead of SLDB arrays for easier and more efficient timestamp management

In [1]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# uncomment the following line for compatibility with TensorFlow 1.15 (on GCP)
# import tensorflow.compat.v1 as tf
# uncomment the following line for TensorFlow 2.X (local execution)
import tensorflow as tf

# forecast model was saved in TensorFlow 1.15
# but, in order to make predictions locally, has to be loaded with TensorFlow 2
from tensorflow.saved_model import load

In [2]:
# symmetric mean absolute percentage error
def symmetric_mean_absolute_percentage_error(targets, predictions):
    '''
    predictions: a list with the predicted values
    targets: a list with the actual values
    '''
    import numpy as np
    # lists to NumPy arrays
    targets, predictions = np.array(targets), np.array(predictions)
    # verify predictions and targets have the same shape
    if predictions.shape == targets.shape:
            return(np.sum(2*np.abs(predictions - targets) /
                          (np.abs(targets) + np.abs(predictions)))/predictions.shape[0])

In [3]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [4]:
# converts a set of tensors to a feature dict to a serialized example to pass it
# to the prediction function of the saved model 
def input_tensors_to_serialized_example(encoder_input_float_tensor,
                                        decoder_input_float_tensor,
                                        sequential_id_float_tensor):
    # first, pass the float tensors to NumPy array, then flatten them
    encoder_input_flat_array = encoder_input_float_tensor.numpy().flatten()
    decoder_input_flat_array = decoder_input_float_tensor.numpy().flatten()
    sequential_id_flat_array = sequential_id_float_tensor.numpy().flatten()
    
    # second, build the protobuffer example
    example = tf.train.Example(
        # features within the example
        features=tf.train.Features(
            # feature definition
            feature={
                'encoder_input': _float_feature_from_list_of_values(encoder_input_flat_array),
                'decoder_input': _float_feature_from_list_of_values(decoder_input_flat_array),
                'sequential_id': _float_feature_from_list_of_values(sequential_id_flat_array)
            }
        )
    )    
    # third, serialize the example dictionary to a string
    serialized_example = example.SerializeToString()
    # fourth, wrap the serialized example as a NumPy-string array
    numpy_example = np.array(serialized_example, dtype='S')
    # fifth, wrap the NumPy-string array as a string tensor
    serialized_example = tf.convert_to_tensor(numpy_example)

    return serialized_example

In [5]:
PROJECT_ROOT = '/home/developer/gcp/cbidmltsf'

In [16]:
# during batch prediction, the model identifier is obtained via Abseil Flags
# remember this notebook is based on local execution,
# therefore model directory must be downloaded from GS before running the notebook
# model_id = 'BSCTRFM_TPU_070'

In [17]:
# during batch prediction, the execution identifier is obtained via Abseil Flags
# execution = 0

In [18]:
# during batch prediction, the SLDB identifier is obtained via Abseil Flags
# THE SLDB FOR INFERENCE MUST BE THE SAME USED FOR TRAINING! (THE ONE SETUP IN THE CONFIGURATION FILE)
sldb_id = 'PEMS-SF_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX'

In [19]:
# during batch prediction, the dataset name is obtained via Abseil Flags
dataset = 'test'

In [20]:
# define a forecast window to guide the iterative prediction process
# start with a hourly, day-ahead process
forecast_window = 24

In [21]:
# ADD AN INFERENCE IDENTIFIER, BECAUSE FOR TRANSFORMER-BASED MODELS, DIFFERENT INFERENCES
# CAN BE PRODUCED FROM A SINGLE SAVED MODEL (USUALLY TO PRODUCE DIFFERENT FORECAST WINDOWS)
# during batch prediction, the inference identifier should be obtained via Abseil Flags
inference = '{:03d}'.format(forecast_window)

In [22]:
# build a path to the SLDB json file
data_dir = '{}/{}/{}'.format(PROJECT_ROOT, 'sldbs', sldb_id)

# then get the ts_identifier from the json file in the sldb directory
sldb_json_file = '{}/sldb.json'.format(data_dir)

In [23]:
# open the json file
with open(sldb_json_file, 'r') as inputfile:
    sldb_dict = json.load(inputfile)

In [26]:
# and get the time series identifier
ts_identifier = sldb_dict['ts']
ts_identifier

'PEMS-SF_SEPARATED_FULL'

In [27]:
m = 168
m

168

In [28]:
t = 168
t

168

In [ ]:
# avoid using iterrows for Pandas dataframes (it is an anti-pattern!)

# use an integer range over the dataframe rows to integer-locate data

In [42]:
# get the range over the number of rows
saved_models_df_range = range(0, saved_models_df.shape[0])

# iterate over a given slice of integer locations
for integer_location in saved_models_df_range[:1]:
    print(
        # get the index as the row.name
        saved_models_df.iloc[integer_location].name,
        saved_models_df.iloc[integer_location]['model_id'],
        saved_models_df.iloc[integer_location]['execution'],
        saved_models_df.iloc[integer_location]['train_steps']
    )

1637776739 BSCTRFM_TPU_075 0 5000


In [43]:
encoder_input_columns = [
    'occupancy_scaled',
    'sin_hours_from_start',
    'cos_hours_from_start',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    # 'sin_day_month',
    # 'cos_day_month',
    # 'sin_day_year',
    # 'cos_day_year'
]

In [44]:
decoder_input_columns = encoder_input_columns

In [45]:
id_columns = ['sequential_id']

In [46]:
# get and test the starting indexes list to produce 7 hourly-day-ahead prediction sets for benchmarking

dataset_row_indexes_list = [0, 24, 48, 72, 96, 120, 144]

In [47]:
data_folder = '{}/datasets/traffic/PEMS-SF'.format(PROJECT_ROOT)
data_folder

'/home/developer/gcp/cbidmltsf/datasets/traffic/PEMS-SF'

In [48]:
# get the station identifies list from the dataset folder

In [49]:
def process_list(s, variable_type=int, delimiter=None):
    """Parses a line in the PEMS format to a list."""
    if delimiter is None:
      l = [
          variable_type(i) for i in s.replace('[', '').replace(']', '').split()
      ]
    else:
      l = [
          variable_type(i)
          for i in s.replace('[', '').replace(']', '').split(delimiter)
      ]

    return l

In [50]:
def read_single_list(filename):
    """Returns single list from a file in the PEMS-custom format."""
    with open(os.path.join(data_folder, filename), 'r') as dat:
        l = process_list(dat.readlines()[0])
    return l

In [51]:
station_ids = [id for id in read_single_list('stations_list')]
len(station_ids)

963

In [127]:
saved_models = [
    # '1637776739',
    # '1637777357',
    '1637778389',
    '1637778735',
    '1637780410',
    '1637777047',
    '1637777674',
    '1637778065',
    '1637779049',
    '1637779397',
    '1637779753',
    '1637780062'
]

In [128]:
model_ids = [
    # 'BSCTRFM_TPU_075',
    # 'BSCTRFM_TPU_075',
    'BSCTRFM_TPU_075',
    'BSCTRFM_TPU_075',
    'BSCTRFM_TPU_075',
    'BSCTRFM_TPU_075',
    'BSCTRFM_TPU_075',
    'BSCTRFM_TPU_075',
    'BSCTRFM_TPU_075',
    'BSCTRFM_TPU_075',
    'BSCTRFM_TPU_075',
    'BSCTRFM_TPU_075'
]

In [129]:
executions = [
    # 0,
    # 0,
    0,
    0,
    0,
    1,
    1,
    1,
    1,
    1,
    1,
    1
]

In [130]:
for saved_model_id, model_id, execution in zip(saved_models, model_ids, executions):
    print(saved_model_id, model_id, execution)

1637778389 BSCTRFM_TPU_075 0
1637778735 BSCTRFM_TPU_075 0
1637780410 BSCTRFM_TPU_075 0
1637777047 BSCTRFM_TPU_075 1
1637777674 BSCTRFM_TPU_075 1
1637778065 BSCTRFM_TPU_075 1
1637779049 BSCTRFM_TPU_075 1
1637779397 BSCTRFM_TPU_075 1
1637779753 BSCTRFM_TPU_075 1
1637780062 BSCTRFM_TPU_075 1


## skip the following code to avoid inference process

In [131]:
# a second inference identifier to run more than one inference
# on the same combination model_id, execution, inference;
# to produce different inference processes because predict_fn is stochastic
event = 0

for saved_model_id, model_id, execution in zip(saved_models, model_ids, executions):
    # print(saved_model_id, model_id, execution)
    
    # use model identifier and execution number to build the model directory string
    model_dir = '{}_{:02d}'.format(model_id, execution)

    # get the path to the saved model main directory
    saved_model_path = '{}/{}/{}/export/exporter'.format(PROJECT_ROOT,
                                                         'models',
                                                         model_dir)

    # build the full path for the latest saved model dir
    export_dir = '{}/{}'.format(saved_model_path, saved_model_id)
    print ('Exported model path is {}'.format(export_dir))

    # load the saved model and the prediction function
    imported = load(export_dir=export_dir, tags='serve')
    predict_fn = imported.signatures["serving_default"]
    
    
    # use a dictionary to remain the code consistent with the SLDB building process
    # most of the times, only ts['test'] will be used for inference
    # however, ts['eval'] might also be used, as it have not really been seen by training process
    # (no modification in model training resulted from evaluation stage)

    ts_test = dict()

    # a dictionary to store predictions detail dataframe per station id
    predictions_detail = dict()
    
    for station_id in station_ids:

        # read the time series for the current sensor station
        ts_test[station_id] = pd.read_pickle('{}/test/ST_{}.pkl'.format(data_dir, station_id))

        # report start and end timestamp for the loaded time series
        print('Loaded test time series for {}, which spans from index {} to index {}'.\
             format(station_id,
                    str(ts_test[station_id].index[:1][0]),
                    str(ts_test[station_id].index[-1:][0])))

        scaler_type = 'min_max'

        if scaler_type == 'min_max':
            # build a path to the scaler of the time series
            scaler_path = '{}/scalers/min_max_{}.save'.format(data_dir, station_id)
            # and load it
            scaler = joblib.load(scaler_path)


        '''
        if scaler_type == 'standard':
            # build a path to the scaler of the time series
            scaler_path = '{}/scalers/standard_{}.save'.format(data_dir, station_id)
            # and load it
            scaler = joblib.load(scaler_path)
        '''    


        # a columns list for the predictions dataframe
        pred_df_columns = ['model_id',
                           'execution',
                           'dataset',
                           'inference',
                           'station_id',
                           # no date information in the traffic dataset, then
                           # replace string_timestamps with string_indexes
                           # 'string_timestamps',
                           'string_indexes',
                           'predictions',
                           'targets']

        # build the predictions dataframe as a key-value pair of the dictionary
        predictions_detail[station_id] = pd.DataFrame(columns=pred_df_columns)

        # iterate on a set of valid rows of the test dataset
        # starting_point = 0 # based on the inference dataset
        # span = 1 + 6*24 # number of days in the test dataset, expressed in hours

        # MAKE THE PREDICTION SETS FOR BENCHMARKING ONLY! 
        # dataset_row_indexes_list = starting_point + np.arange(span)
        dataset_row_indexes_list = [0, 24, 48, 72, 96, 120, 144]

        for start_index in dataset_row_indexes_list:

            # define first prediction interval with start- and end-index
            # given the interval time_series[start_index:end_index]
            # the conditioning range is the union of the encoder-input and the decoder-input
            # and the prediction range is only the last lecture in the interval,
            # by means of a recursive inference process
            # on each step the last prediction is added to the decoder input
            # and the prediction range grows one step into the future

            # get the end-index of this recursive inference interval
            end_index = start_index + (m + t)

            # initialize a list to store recurrent predictions for this interval
            predictions_list = list()

            for i in np.arange(forecast_window):

                # build the inference interval as a sub-series of the dataset
                # the following slicing operation is equivalent to direct slicing as in:
                # sub_series = ts_test[station_id][start_index + i : end_index + i]
                sub_series = ts_test[station_id].iloc[start_index + i : end_index + i]

                # important: build sources as copies of the sub-series (and therefore of the global time series)
                # to avoid overwriting the original dataset

                # the encoder input source
                encoder_input = sub_series[encoder_input_columns][:m].copy()

                # the decoder input source
                # decoder_input = sub_series[m-1:-1].copy()
                decoder_input = sub_series[decoder_input_columns][m-1:m-1+t].copy()

                # the id (integer) for the sensor station
                id_input = sub_series[id_columns][:1].copy()

                # on first step (i=0), the decoder input carries only true values
                # and the predictions list is empty
                # on subsequent steps, the decoder input includes all previous predictions
                # (stored in the predictions list)
                if i > 0:
                    decoder_input['occupancy_scaled'][-i:] = predictions_list

                # the target source, for metrics calculation
                # the first part of the sub-series is the encoder input, and
                # the second part of the sub-series is the target (only the variable column!)
                target = sub_series['occupancy_scaled'][m:].copy()

                # build source tensors from the sub-series    
                encoder_input_tensor = tf.expand_dims(encoder_input, axis=0)
                decoder_input_tensor = tf.expand_dims(decoder_input, axis=0)
                id_tensor = tf.expand_dims(id_input, axis=0)

                # make input example for the prediction function
                input_example = input_tensors_to_serialized_example(encoder_input_tensor,
                                                                    decoder_input_tensor,
                                                                    id_tensor)

                # get the output of the prediction function as a dictionary
                predict_output_dict = predict_fn(input_example)

                # get the prediction output tensor
                predict_output_tensor = predict_output_dict['forecast']
                
                # get the most recent prediction
                most_recent_prediction = predict_output_tensor[0, :, 0].numpy()[-1]

                # append the most recent prediction timestep to the predictions list
                predictions_list.append(most_recent_prediction)

                # pass the predictions list to an array
                # current_predictions_array = np.array(predictions_list).reshape(-1, 1)
                # get the targets vector to be compared with the current predictions array
                # current_targets = np.array(target[-i-1:]).reshape(-1, 1)

                # calculate SMAPE on the rescaled variable
                # rescaled_predictions = min_max_scaler.inverse_transform(current_predictions_array)
                # rescaled_targets = min_max_scaler.inverse_transform(current_targets)

                # current_smape = symmetric_mean_absolute_percentage_error(
                #     rescaled_targets, rescaled_predictions)
                # print('On row {}, SMAPE for the first {} rescaled prediction(s) is {}'.format(start_index,
                #                                                                               i + 1,
                #                                                                               current_smape))        

            # iterative predictions over the forecast window reside in predictions_list
            # convert list to array, then expand feature dimension with value 1
            predicted_values = np.array(predictions_list).reshape(-1, 1)

            # inverse-scale predictions
            rescaled_predicted_values = scaler.inverse_transform(predicted_values)

            # and the true values remain in the prediction tensor, pass them to a NumPy array
            # for the true values array, expand feature dimension with value 1
            true_values = np.array(target[-i-1:]).reshape(-1, 1)

            # inverse-scale true values
            rescaled_true_values = scaler.inverse_transform(true_values)

            # a temporary dataframe built from the data in the current row
            df = pd.DataFrame(columns=pred_df_columns)
            df['model_id'] = [model_id]
            df['execution'] = [execution]
            df['dataset'] = [dataset]
            df['inference'] = [inference]
            df['station_id'] = [station_id]
            # df['string_timestamps']= [pd.to_datetime(target.index[-i-1:]).astype(str).tolist()]
            df['string_indexes']= [target.index[-i-1:].astype(str).tolist()]
            df['predictions'] = [np.squeeze(rescaled_predicted_values).tolist()]
            df['targets'] = [np.squeeze(rescaled_true_values).tolist()]

            # calculate mean absolute error and normalized deviation
            mae = mean_absolute_error(rescaled_true_values, rescaled_predicted_values)
            df['mae'] = mae

            true_values_average = np.mean(rescaled_true_values)
            df['nd'] = mae/true_values_average

            # calculate root mean squared error and normalized root mean squared error
            rmse = sqrt(mean_squared_error(rescaled_true_values, rescaled_predicted_values))
            df['rmse'] = rmse
            df['nrmse'] = rmse/true_values_average

            df['smape'] = symmetric_mean_absolute_percentage_error(rescaled_true_values,
                                                                   rescaled_predicted_values)

            # append the temporary dataframe to the predictions detail dataframe
            predictions_detail[station_id] = pd.concat([predictions_detail[station_id], df])


        # reset the index of final dataframe, once all of its rows (dataset) have been processed
        predictions_detail[station_id] = predictions_detail[station_id].reset_index(drop=True)


        # hard-wired switcher to persist dataframe
        persist_detail = True

        if persist_detail:
            # build a path to persist the dataframe to database/predictions_detail/
            detail_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}_{}_{:02d}.pkl'.format(
                PROJECT_ROOT,
                'database',
                'predictions_detail',
                model_id,
                execution,
                saved_model_id,
                station_id,
                inference,
                event)

            # persist the Pandas dataframe to database/predictions_detail/
            predictions_detail[station_id].to_pickle(detail_pickle_path)
            print(
                'Persisted predictions detail of {}_{:02d}_{}_{}_{}_{:02d}'.format(model_id,
                                                                                   execution,
                                                                                   saved_model_id,
                                                                                   station_id,
                                                                                   inference,
                                                                                   event)
            )

Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_075_00/export/exporter/1637778389
Loaded test time series for 400000, which spans from index 3648 to index 4150
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400000_024_00
Loaded test time series for 400001, which spans from index 14206 to index 14708
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400001_024_00
Loaded test time series for 400009, which spans from index 24764 to index 25266
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400009_024_00
Loaded test time series for 400010, which spans from index 35322 to index 35824
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400010_024_00
Loaded test time series for 400015, which spans from index 45880 to index 46382
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400015_024_00
Loaded test time series for 400017, which spans from index 56438 to index 56940
Persisted predictions detail of BSCT

Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400113_024_00
Loaded test time series for 400115, which spans from index 552664 to index 553166
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400115_024_00
Loaded test time series for 400116, which spans from index 563222 to index 563724
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400116_024_00
Loaded test time series for 400118, which spans from index 573780 to index 574282
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400118_024_00
Loaded test time series for 400122, which spans from index 584338 to index 584840
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400122_024_00
Loaded test time series for 400124, which spans from index 594896 to index 595398
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400124_024_00
Loaded test time series for 400125, which spans from index 605454 to index 605956
Persisted predictions detail of BSCTRFM_TPU_075_00_1

Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400212_024_00
Loaded test time series for 400213, which spans from index 1101680 to index 1102182
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400213_024_00
Loaded test time series for 400214, which spans from index 1112238 to index 1112740
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400214_024_00
Loaded test time series for 400216, which spans from index 1122796 to index 1123298
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400216_024_00
Loaded test time series for 400218, which spans from index 1133354 to index 1133856
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400218_024_00
Loaded test time series for 400221, which spans from index 1143912 to index 1144414
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400221_024_00
Loaded test time series for 400222, which spans from index 1154470 to index 1154972
Persisted predictions detail of BSCTRFM_

Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400309_024_00
Loaded test time series for 400312, which spans from index 1650696 to index 1651198
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400312_024_00
Loaded test time series for 400313, which spans from index 1661254 to index 1661756
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400313_024_00
Loaded test time series for 400314, which spans from index 1671812 to index 1672314
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400314_024_00
Loaded test time series for 400315, which spans from index 1682370 to index 1682872
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400315_024_00
Loaded test time series for 400316, which spans from index 1692928 to index 1693430
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400316_024_00
Loaded test time series for 400317, which spans from index 1703486 to index 1703988
Persisted predictions detail of BSCTRFM_

Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400418_024_00
Loaded test time series for 400422, which spans from index 2199712 to index 2200214
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400422_024_00
Loaded test time series for 400423, which spans from index 2210270 to index 2210772
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400423_024_00
Loaded test time series for 400425, which spans from index 2220828 to index 2221330
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400425_024_00
Loaded test time series for 400428, which spans from index 2231386 to index 2231888
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400428_024_00
Loaded test time series for 400429, which spans from index 2241944 to index 2242446
Persisted predictions detail of BSCTRFM_TPU_075_00_1637778389_400429_024_00
Loaded test time series for 400430, which spans from index 2252502 to index 2253004
Persisted predictions detail of BSCTRFM_

...

Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401603_024_00
Loaded test time series for 401605, which spans from index 7816568 to index 7817070
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401605_024_00
Loaded test time series for 401606, which spans from index 7827126 to index 7827628
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401606_024_00
Loaded test time series for 401608, which spans from index 7837684 to index 7838186
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401608_024_00
Loaded test time series for 401609, which spans from index 7848242 to index 7848744
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401609_024_00
Loaded test time series for 401610, which spans from index 7858800 to index 7859302
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401610_024_00
Loaded test time series for 401611, which spans from index 7869358 to index 7869860
Persisted predictions detail of BSCTRFM_

Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401683_024_00
Loaded test time series for 401684, which spans from index 8365584 to index 8366086
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401684_024_00
Loaded test time series for 401686, which spans from index 8376142 to index 8376644
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401686_024_00
Loaded test time series for 401688, which spans from index 8386700 to index 8387202
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401688_024_00
Loaded test time series for 401689, which spans from index 8397258 to index 8397760
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401689_024_00
Loaded test time series for 401691, which spans from index 8407816 to index 8408318
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401691_024_00
Loaded test time series for 401692, which spans from index 8418374 to index 8418876
Persisted predictions detail of BSCTRFM_

Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401808_024_00
Loaded test time series for 401809, which spans from index 8914600 to index 8915102
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401809_024_00
Loaded test time series for 401810, which spans from index 8925158 to index 8925660
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401810_024_00
Loaded test time series for 401811, which spans from index 8935716 to index 8936218
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401811_024_00
Loaded test time series for 401816, which spans from index 8946274 to index 8946776
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401816_024_00
Loaded test time series for 401817, which spans from index 8956832 to index 8957334
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401817_024_00
Loaded test time series for 401818, which spans from index 8967390 to index 8967892
Persisted predictions detail of BSCTRFM_

Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401913_024_00
Loaded test time series for 401914, which spans from index 9463616 to index 9464118
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401914_024_00
Loaded test time series for 401917, which spans from index 9474174 to index 9474676
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401917_024_00
Loaded test time series for 401918, which spans from index 9484732 to index 9485234
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401918_024_00
Loaded test time series for 401923, which spans from index 9495290 to index 9495792
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401923_024_00
Loaded test time series for 401924, which spans from index 9505848 to index 9506350
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_401924_024_00
Loaded test time series for 401925, which spans from index 9516406 to index 9516908
Persisted predictions detail of BSCTRFM_

Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_402068_024_00
Loaded test time series for 402069, which spans from index 10012632 to index 10013134
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_402069_024_00
Loaded test time series for 402070, which spans from index 10023190 to index 10023692
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_402070_024_00
Loaded test time series for 402071, which spans from index 10033748 to index 10034250
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_402071_024_00
Loaded test time series for 402072, which spans from index 10044306 to index 10044808
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_402072_024_00
Loaded test time series for 402077, which spans from index 10054864 to index 10055366
Persisted predictions detail of BSCTRFM_TPU_075_01_1637780062_402077_024_00
Loaded test time series for 402081, which spans from index 10065422 to index 10065924
Persisted predictions detail